In [4]:
import sys

sys.path.append("..")

In [5]:
import pandas as pd

from tqdm import tqdm
from classifier.preprocessor import Preprocessor

In [7]:
tqdm.pandas()
pd.set_option("max_colwidth", None)

In [8]:
df = pd.read_csv("../data/train.csv")

In [27]:
preprocessor = Preprocessor(
    do_stem=False,
    remove_stop_words=True, 
    black_list=r"\b.*id.*\b",
    remove_single_letter_token=True,
    max_token_length=25,
)

In [28]:
df["text_preprocessed"] = df["text"].progress_apply(preprocessor._preprocess)

100% 38740/38740 [00:14<00:00, 2594.15it/s]


In [29]:
df[["text", "text_preprocessed"]].sample(10)

,text,text_preprocessed
18689,Жизнь на состоит из того что с вами происходит и на 9⃣0⃣ из того как вы на это реагируете.,жизнь состоит вами происходит 9⃣0⃣ это реагируете
36686,БАЛКАРСКАЯ МУДРОСТЬ Къыш дегенинг бир бетли Жаз дегенинг минг бетли. Къыш тасхасын жаз ачар. У зимы один цвет У весны тысяча. Тайны зимы раскрывает весна. Къышхыда хапар ахшы Жазгъыда сафар ахшы. Зимой хороша беседа Весной путешествие. Жаз хауаны къуту бар Жай хауаны оту бар. В весеннем воздухе есть дух В летнем воздухе огонь. Къыш умутунг жай толур. Къыш юйретир жай ишлетир. Задуманное зимой исполняется летом. Зима учит лето заставляет работать. Кюз арты ой л ар ючюн. Жаз башы кезге асыу Кюз арты келге асыу. Поздняя осень чтобы думать. Начало весны утешение для глаз Конец осени утешение для души. Источник Ёзден Адет этический кодекс карачаево балкарского народа. Автор фотографий Кюз арты 2022 год. Структуры древних знаний безенгийских гор.,балкарская мудрость къыш дегенинг бир бетли жаз дегенинг минг бетли къыш тасхасын жаз ачар зимы цвет весны тысяча тайны зимы раскрывает весна къышхыда хапар ахшы жазгъыда сафар ахшы зимой хороша беседа весной путешествие жаз хауаны къуту бар жай хауаны оту бар весеннем воздухе дух летнем воздухе огонь къыш умутунг жай толур къыш юйретир жай ишлетир задуманное зимой исполняется летом зима учит лето заставляет работать кюз арты ой ар ючюн жаз башы кезге асыу кюз арты келге асыу поздняя осень думать начало весны утешение глаз конец осени утешение души источник езден адет этический кодекс карачаево балкарского народа автор фотографий кюз арты год структуры древних знаний безенгийских гор
12542,Константин Козловский отметился первым шатаутом в Кубке Салея 2022 Вратарь сделал несколько ярких спасений и отразил 30 бросков хоккеистов Металлурга. Локомотив обыграл действующего чемпиона страны со счетом 1 0. Следующий матч железнодорожники проведут 5 августа против Гомеля.,константин козловский отметился первым шатаутом кубке салея вратарь сделал несколько ярких спасений отразил бросков хоккеистов металлурга локомотив обыграл действующего чемпиона страны счетом следующий матч железнодорожники проведут августа против гомеля
14858,В субботу мы играем на полигоне Теплицы 33 Все подробности по игре в этом сообщении Поиск попутчиков на СУББОТУ Аренда страйкбольного снаряжения tokentokenoid Правила ОБЯЗАТЕЛЬНЫЕ для соблюдения на полигоне Теплицы Гранаты ТОЛЬКО СЕРТИФИЦИРОВАННЫЕ любые самопалы запрещены. Пристрелка приводов около машин СТРОГО ЗАПРЕЩЕНА. Стрельба в поле в сторону молодых деревьев СТРОГО ЗАПРЕЩЕНА. Стрелять из приводов можно только на игровой территории 33 Дымы ЗАПРЕЩЕНЫ 33 На Теплицах самодельные гранаты ЗАПРЕЩЕНЫ 33 На другие полигоне самоделки только с внутренним горением петарды внутри корпуса гранаты.,субботу играем полигоне теплицы подробности игре сообщении поиск попутчиков субботу аренда страйкбольного снаряжения правила обязательные соблюдения полигоне теплицы гранаты сертифицированные любые самопалы запрещены пристрелка приводов около машин строго запрещена стрельба поле сторону молодых деревьев строго запрещена стрелять приводов игровой территории дымы запрещены теплицах самодельные гранаты запрещены другие полигоне самоделки внутренним горением петарды внутри корпуса гранаты
15360,Самый ожидаемый турнир года в UFC покажут в кино. Это лучше любого экшена Два титульника. Главный бой в карьере Махачева который может вернуть чемпионский пояс в Россию. И Стерлинг с Диллашоу в драке за титул в легчайшем весе Уже дикий адреналин. А ведь будет еще Петр Ян против дерзкого Шона О’Мэлли Матч ТВ вместе с объединенной сетью кинотеатров Синема Парк и Формула Кино покажет UFC 280 на больших экранах. Вот где ты точно увидишь все хайкики и борьбу в мельчайших деталях Москва Санкт Петербург Тюмень Челябинск Набережные Челны Уфа Волгоград Нижний Новгород Екатеринбург Вологда Белгород Саратов Воронеж Ульяновск Сургут Ставрополь Тула Пермь Ижевск Рязань и Сочи – ищи кинотеатр в своем городе и покупай билет пока не расхватали,са

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [40]:
vectorizer = TfidfVectorizer(analyzer="char_wb", ngram_range=(2, 5), max_features=10_000)

In [41]:
vectorizer.fit(df.text_preprocessed)

TfidfVectorizer(analyzer='char_wb', max_features=10000, ngram_range=(2, 5))

In [42]:
len(vectorizer.vocabulary_)

10000

In [43]:
from sklearn.linear_model import LogisticRegression

In [44]:
model = LogisticRegression()

In [45]:
model.fit(
    vectorizer.transform(df.text_preprocessed),
    df.category
)

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

## Submit

In [46]:
hidden = pd.read_csv("../data/test.csv")

In [47]:
hidden["text_preprocessed"] = hidden["text"].progress_apply(preprocessor._preprocess)

100% 26260/26260 [00:10<00:00, 2557.53it/s]


In [61]:
hidden_features = vectorizer.transform(hidden.text_preprocessed.tolist())

In [81]:
hidden_predictions = model.predict_proba(hidden_features)

In [87]:
hidden["predictions"] = list(hidden_predictions)

In [88]:
hidden["category"] = hidden["predictions"].apply(lambda predictions: model.classes_[predictions.argmax()])

In [89]:
hidden["score"] = hidden["predictions"].apply(max)

In [92]:
hidden[hidden.score >= 0.8][["oid", "category"]].to_csv("../data/submission.csv", index=False)